In [1]:
import sys
sys.path.append("/home/149/ab8992/tasman-tides/")
import xarray as xr
import ttidelib as tt
import scipy
import cmocean
import os
from pathlib import Path
cmap = cmocean.cm.dense_r
import matplotlib
import matplotlib.pyplot as plt
earth_cmap = matplotlib.colormaps["gist_earth"]
from datetime import timedelta
import warnings
warnings.simplefilter("ignore")
# import filtering
import numpy as np
import dask
dask.config.set({'logging.distributed': 'error'})
from dask.distributed import Client,default_client
import xrft


client = tt.startdask(nthreads=1,n_workers = 20)

client

<Client: 'tcp://127.0.0.1:42055' processes=20 threads=20, memory=416.00 GiB>


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/40717/status,
Dashboard: /proxy/40717/status,Workers: 20
Total threads: 20,Total memory: 416.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:42055,Workers: 20
Dashboard: /proxy/40717/status,Total threads: 20
Started: Just now,Total memory: 416.00 GiB
Comm: tcp://127.0.0.1:46243,Total threads: 1
Dashboard: /proxy/44905/status,Memory: 20.80 GiB
Nanny: tcp://127.0.0.1:44139,


In [2]:
tmpstorage = os.getenv('PBS_JOBFS')
tmpstorage = "/scratch/nm03/ab8992/test"
outputdir = "/scratch/nm03/ab8992/test/outputs"
def DirectionalFilter(data,dim = "xb"):
    """
    Fourier filter into forward and backward propagating signals
    """

    
    if "lat" in data.coords:
        data = data.drop(["lat","lon"])
    import xrft
    FT = xrft.fft(
        data,dim = ["time",dim]
    )

    forward = np.real(xrft.ifft(
        FT.where((FT[f"freq_{dim}"] > 0) & (FT.freq_time > 0), 0) +
          FT.where((FT[f"freq_{dim}"] < 0) & (FT.freq_time < 0), 0),
        dim = ["freq_time",f"freq_{dim}"]
    ))

    backward = np.real(xrft.ifft(
        FT.where((FT[f"freq_{dim}"] < 0) & (FT.freq_time > 0), 0) + 
        FT.where((FT[f"freq_{dim}"] > 0) & (FT.freq_time < 0), 0),
        dim = ["freq_time",f"freq_{dim}"]
    ))

    return xr.merge([forward.rename(f"{data.name}_forward"),backward.rename(f"{data.name}_backward")]).assign_coords({"time":data.time,dim:data[dim]})
def laplacian(data):
    out = data.u.bfill("xb",limit = 4).ffill("xb",limit = 4).differentiate("xb").differentiate("xb")
    out +=data.v.bfill("yb",limit = 4).ffill("yb",limit = 4).differentiate("yb").differentiate("yb")
    return out

def _scipy_integrate(data, zl):
    """
    Helper function to perform cumulative trapezoidal integration along the 'zl' axis.
    """
    return scipy.integrate.cumulative_trapezoid(data, x=zl, initial=0)

def scipy_integrate(data):
    """
    Wrapper function to apply cumulative trapezoidal integration along the 'zl' axis.
    """
    zl = data.zl.values  # Extract 'zl' coordinate values
    return np.apply_along_axis(_scipy_integrate, data.get_axis_num('zl'), data, zl=zl)

def save_temporary(expt,t0,outputdir):
    """
    Open all the datasets that we need for this experiment. Save some of the simple stuff, and save the modal decompossitions to temporary storage chunked by mode for further processing.
    """
    with (
            xr.open_mfdataset(f"/g/data/nm03/ab8992/postprocessed/{expt}/bandpassed/t0-{t0}/Filtered*.nc",decode_times = False) as filtered,
            tt.collect_data(expt,rawdata=["u","v","ahh"],timerange = (filtered.time.values[0],filtered.time.values[-1])).isel(zl = slice(0,96)) as raw

    ):
        if "40" in expt: #! vmodes have failed for some times. For now, fix vmodes to match for each run
            vmodes = xr.open_dataset(f"/g/data/nm03/ab8992/postprocessed/{expt}/vertical_eigenfunctions/vmode-t0-{4216}.nc",decode_times = False,chunks = {"mode":1}).isel(zl = slice(0,96))
        else:
            vmodes = xr.open_dataset(f"/g/data/nm03/ab8992/postprocessed/{expt}/vertical_eigenfunctions/vmode-t0-{22000}.nc",decode_times = False,chunks = {"mode":1}).isel(zl = slice(0,96))

            
        # os.remove(f"{tmpstorage}/*.nc")
        vmodes = vmodes.assign_coords({"zl":filtered.zl})
        ymin = vmodes.yb[0].values - 0.0001 ## This ensures random numerical changes of 1e-16 to axis values don't cause issues... 
        ymax = vmodes.yb[-1].values + 0.0001
        raw = raw.sel(yb = slice(ymin,ymax)).assign_coords(vmodes.isel(mode = 0).coords).drop_vars(["lat","lon"])
        filtered = filtered.sel(yb = slice(ymin,ymax)).assign_coords(vmodes.isel(mode = 0).coords)


        ## Here all variables are dask arrays. One by one, compute what we want and save to PBS temporart storage 

        for i in range(len(vmodes.mode.values)):
            # print("Saving mode ",i)
            (vmodes.U * filtered.u).fillna(0).integrate("zl").rename("u").isel(mode = [i]).to_netcdf(f"{tmpstorage}/u_{i}.nc")
            (vmodes.U * filtered.v).fillna(0).integrate("zl").rename("v").isel(mode = [i]).to_netcdf(f"{tmpstorage}/v_{i}.nc")
            (vmodes.W * filtered.rho).fillna(0).integrate("zl").rename("rho").isel(mode = [i]).to_netcdf(f"{tmpstorage}/rho_{i}.nc")

        #! OLD VERSION: I was missing the depth in the cumsum. this should be a proper integral, 
        #! so needed to multiply by cell thicknesses
        #! (
        #!     vmodes.U * vmodes.W.cumsum("zl") * 9.8
        #!     ).fillna(0).integrate("zl").rename("p_coeff").to_netcdf(f"{tmpstorage}/p_coeff.nc")        

        #! Updated: Now use the scipy integrate function to do cumsum correctly, retaining cell thicknesses
        (
            vmodes.U * scipy_integrate(vmodes.W) * 9.8
            ).fillna(0).integrate("zl").rename("p_coeff").to_netcdf(f"{tmpstorage}/p_coeff.nc") 


        ## While we have files open, might as well save some other simple stuff, like vorticity in top 20 layers, total raw KE, both bc and bt.
        # print("Save vorticity...")

        ((raw.v.differentiate("xb") - raw.u.differentiate("yb")).fillna(0).mean("time").sel(zl = slice(0,200)).integrate("zl") / 200).rename("vorticity").to_netcdf(f"{outputdir}/raw_vorticity.nc")

        print("Save raw KE...")
        (raw.u**2 + raw.v**2).fillna(0).integrate("zl").mean("time").rename("raw_KE_total").to_netcdf(f"{outputdir}/raw_ke_total.nc")
        # print("Save raw KE bt...")
        (
            (raw.u.fillna(0).integrate("zl")**2 + raw.v.fillna(0).integrate("zl")**2) / raw.bathy
        ).mean("time").rename("raw_KE_bt").to_netcdf(f"{outputdir}/raw_ke_bt.nc")
        # print("Save raw dissipation...")
        (
            (raw.ahh * laplacian(raw)**2).mean("time") / (1e6)
        ).fillna(0).integrate("zl").rename("raw_dissipation").to_netcdf(f"{outputdir}/raw_dissipation.nc")

        # print("Save filtered KE...")
        (filtered.u**2 + filtered.v**2).fillna(0).integrate("zl").mean("time").rename("filtered_KE_total").to_netcdf(f"{outputdir}/filtered_ke_total.nc")
        # print("Save filtered KE bt...")
        (
            (filtered.u.fillna(0).integrate("zl")**2 + filtered.v.fillna(0).integrate("zl")**2) / raw.bathy
        ).mean("time").rename("filtered_KE_bt").to_netcdf(f"{outputdir}/filtered_ke_bt.nc")
        print("Save filtered dissipation...")
        (
            (raw.ahh * laplacian(filtered)**2).mean("time") / (1e6)
        ).fillna(0).integrate("zl").rename("filtered_dissipation").to_netcdf(f"{outputdir}/filtered_dissipation.nc")        

    return

def EF_from_u_rho(u,rho,p_coeff): #! TODO MAKE SURE THIS WORKS FOR NON FILTERED EF!
    if isinstance(rho, xr.DataArray):
        EF = rho * u * p_coeff
        return EF.rename("EF")

    EF_forward = rho[f"rho_forward"] * u[f"u_forward"] * p_coeff
    EF_backward = rho[f"rho_backward"] * u[f"u_backward"] * p_coeff
    EF_cross = rho[f"rho_backward"] * u[f"u_forward"] * p_coeff
    EF_cross += rho[f"rho_forward"] * u[f"u_backward"] * p_coeff
    return xr.merge([EF_forward.rename("EF_alongbeam_forward"),EF_backward.rename("EF_alongbeam_backward"),EF_cross.rename("EF_alongbeam_xterm")]).mean("time")

def EF_from_v_rho(v,rho,p_coeff): #! TODO MAKE SURE THIS WORKS FOR NON FILTERED EF!
    if isinstance(rho, xr.DataArray):
        EF = rho * v * p_coeff
        return EF.rename("EF")

    EF_forward = rho[f"rho_forward"] * v[f"v_forward"] * p_coeff
    EF_backward = rho[f"rho_backward"] * v[f"v_backward"] * p_coeff
    EF_cross = rho[f"rho_backward"] * v[f"v_forward"] * p_coeff
    EF_cross += rho[f"rho_forward"] * v[f"v_backward"] * p_coeff
    return xr.merge([EF_forward.rename("EF_acrossbeam_forward"),EF_backward.rename("EF_acrossbeam_backward"),EF_cross.rename("EF_acrossbeam_xterm")]).mean("time")

def save_modal(outputdir):
    U = xr.open_mfdataset(f"{tmpstorage}/u_*.nc",combine = "by_coords",decode_times = False).u
    V = xr.open_mfdataset(f"{tmpstorage}/v_*.nc",combine = "by_coords",decode_times = False).v
    rho = xr.open_mfdataset(f"{tmpstorage}/rho_*.nc",combine = "by_coords",decode_times = False).rho
    p_coeff = xr.open_mfdataset(f"{tmpstorage}/p_coeff.nc",combine = "by_coords",decode_times = False).p_coeff

    ## EF unfiltered
    EF_from_u_rho(U,rho,p_coeff).to_netcdf(f"{outputdir}/EF_alongbeam.nc")
    EF_from_v_rho(V,rho,p_coeff).to_netcdf(f"{outputdir}/EF_acrossbeam.nc")

    ## EF filtered
    print("saving EF")
    with EF_from_u_rho(DirectionalFilter(U),DirectionalFilter(rho),p_coeff) as EF:
        EF["EF_alongbeam_forward"].to_netcdf(f"{outputdir}/EF_alongbeam_forward.nc",mode = "w")
        EF["EF_alongbeam_backward"].to_netcdf(f"{outputdir}/EF_alongbeam_backward.nc",mode = "w")
        EF["EF_alongbeam_xterm"].to_netcdf(f"{outputdir}/EF_alongbeam_xterm.nc",mode = "w")
        
    with EF_from_v_rho(DirectionalFilter(V),DirectionalFilter(rho),p_coeff) as EF:
        EF["EF_acrossbeam_forward"].to_netcdf(f"{outputdir}/EF_acrossbeam_forward.nc",mode = "w")
        EF["EF_acrossbeam_backward"].to_netcdf(f"{outputdir}/EF_acrossbeam_backward.nc",mode = "w")
        EF["EF_acrossbeam_xterm"].to_netcdf(f"{outputdir}/EF_acrossbeam_xterm.nc",mode = "w")
    return 

## Overwrite so we can save filtered EF
def save_modal(outputdir):
    U = xr.open_mfdataset(f"{tmpstorage}/u_*.nc",combine = "by_coords",decode_times = False).u
    V = xr.open_mfdataset(f"{tmpstorage}/v_*.nc",combine = "by_coords",decode_times = False).v
    rho = xr.open_mfdataset(f"{tmpstorage}/rho_*.nc",combine = "by_coords",decode_times = False).rho
    p_coeff = xr.open_mfdataset(f"{tmpstorage}/p_coeff.nc",combine = "by_coords",decode_times = False).p_coeff

    ## EF unfiltered
    EF_from_u_rho(U,rho,p_coeff).to_netcdf(f"{outputdir}/EF_alongbeam.nc")
    EF_from_v_rho(V,rho,p_coeff).to_netcdf(f"{outputdir}/EF_acrossbeam.nc")

    ## EF filtered
    with DirectionalFilter(U, dim="xb") as Uf, DirectionalFilter(rho, dim="xb") as rhof, DirectionalFilter(V, dim="xb") as Vf:
        EF_along = EF_from_u_rho(Uf, rhof, p_coeff)
        EF_across = EF_from_v_rho(Vf, rhof, p_coeff)
        KE_forward = (Uf["u_forward"]**2 + Vf["v_forward"]**2).mean("time").rename("KE_forward")
        KE_backward = (Uf["u_backward"]**2 + Vf["v_backward"]**2).mean("time").rename("KE_backward")
        KE_cross = (Uf["u_backward"] * Uf["u_forward"] + Vf["v_backward"] * Vf["v_forward"]).mean("time").rename("KE_cross")

        EF_along["EF_alongbeam_forward"].to_netcdf(f"{outputdir}/EF_alongbeam_forward.nc", mode="w")
        EF_along["EF_alongbeam_backward"].to_netcdf(f"{outputdir}/EF_alongbeam_backward.nc", mode="w")
        EF_along["EF_alongbeam_xterm"].to_netcdf(f"{outputdir}/EF_alongbeam_xterm.nc", mode="w")

        EF_across["EF_acrossbeam_forward"].to_netcdf(f"{outputdir}/EF_acrossbeam_forward.nc", mode="w")
        EF_across["EF_acrossbeam_backward"].to_netcdf(f"{outputdir}/EF_acrossbeam_backward.nc", mode="w")
        EF_across["EF_acrossbeam_xterm"].to_netcdf(f"{outputdir}/EF_acrossbeam_xterm.nc", mode="w")
        KE_forward.to_netcdf(f"{outputdir}/KE_forward.nc", mode="w")
        KE_backward.to_netcdf(f"{outputdir}/KE_backward.nc", mode="w")
        KE_cross.to_netcdf(f"{outputdir}/KE_cross.nc", mode="w")
    return 

In [ ]:
import shutil
expts = ["full-10","beamless-10","smooth-10","beamless-20","smooth-20","full-20","full-40","beamless-40","smooth-40"]
# expts = ["full-40","beamless-40","smooth-40"]

t0_20th = 22000
t0_40th = 4216
t0_80th = 11905
expts = ["full-80"]
for expt in expts:
    outputdir = "/scratch/nm03/ab8992/april25/outputs/"+expt
    tmpstorage = "/scratch/nm03/ab8992/april25/tmpstorage/"+expt
    if os.path.exists(outputdir):
        shutil.rmtree(outputdir)
        shutil.rmtree(tmpstorage)

    if not os.path.exists(outputdir):
        os.makedirs(outputdir)
    if not os.path.exists(tmpstorage):
        os.makedirs(tmpstorage)
    print(expt)
    t0 = t0_20th
    if "40" in expt:
        t0 = t0_40th
    save_temporary(expt,t0,outputdir)
    save_modal(outputdir)
    print("Done with ",expt)
    # clear_output()

full-10
loading u...	done.
loading v...	done.
loading ahh...	done.
Saving mode  0
Saving mode  1
Saving mode  2
Saving mode  3
Saving mode  4
Saving mode  5
Saving mode  6
Saving mode  7


/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/distributed/client.py:3357: UserWarning: Sending large graph of size 42.84 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


Save vorticity...
Save raw KE...
Save raw KE bt...
Save raw dissipation...
Save filtered KE...
Save filtered KE bt...
Save filtered dissipation...


/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/xrft/xrft.py:573: FutureWarning: Default ifft's behaviour (lag=None) changed! Default value of lag was zero (centered output coordinates) and is now set to transformed coordinate's attribute: 'direct_lag'.
  warnings.warn(msg, FutureWarning)
/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/xrft/xrft.py:573: FutureWarning: Default ifft's behaviour (lag=None) changed! Default value of lag was zero (centered output coordinates) and is now set to transformed coordinate's attribute: 'direct_lag'.
  warnings.warn(msg, FutureWarning)
/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/xrft/xrft.py:573: FutureWarning: Default ifft's behaviour (lag=None) changed! Default value of lag was zero (centered output coordinates) and is now set to transformed coordinate's attribute: 'direct_lag'.
  warnings.warn(msg, FutureWarning)
/g/data/hh5/public/

Done with  full-10
beamless-10
loading u...	done.
loading v...	done.
loading ahh...	done.
Saving mode  0
Saving mode  1
Saving mode  2
Saving mode  3
Saving mode  4
Saving mode  5
Saving mode  6
Saving mode  7


...

Done with  beamless-40
smooth-40
loading u...	done.
loading v...	done.
loading ahh...	done.
Saving mode  0
Saving mode  1
Saving mode  2
Saving mode  3
Saving mode  4
Saving mode  5
Saving mode  6
Saving mode  7


/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/distributed/client.py:3357: UserWarning: Sending large graph of size 532.46 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


Save vorticity...
Save raw KE...
Save raw KE bt...
Save raw dissipation...
Save filtered KE...
Save filtered KE bt...
Save filtered dissipation...


/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/xrft/xrft.py:573: FutureWarning: Default ifft's behaviour (lag=None) changed! Default value of lag was zero (centered output coordinates) and is now set to transformed coordinate's attribute: 'direct_lag'.
  warnings.warn(msg, FutureWarning)
/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/xrft/xrft.py:573: FutureWarning: Default ifft's behaviour (lag=None) changed! Default value of lag was zero (centered output coordinates) and is now set to transformed coordinate's attribute: 'direct_lag'.
  warnings.warn(msg, FutureWarning)
/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/xrft/xrft.py:573: FutureWarning: Default ifft's behaviour (lag=None) changed! Default value of lag was zero (centered output coordinates) and is now set to transformed coordinate's attribute: 'direct_lag'.
  warnings.warn(msg, FutureWarning)
/g/data/hh5/public/

Done with  smooth-40


## Iterate over many times

In [ ]:
import shutil
expts = ["full-10","beamless-10","smooth-10","beamless-20","smooth-20","full-20","full-40","beamless-40","smooth-40"]
# expts = ["full-40","beamless-40","smooth-40"]

import warnings
from IPython.display import clear_output
warnings.simplefilter("ignore")


t0_20th = 22000
t0_40th = 4216
for expt in expts:
    print(f"Starting {expt}" )

    times = os.listdir(f"/g/data/nm03/ab8992/postprocessed/{expt}/bandpassed/")
    for t0 in times:
        outputdir = f"/scratch/nm03/ab8992/april-manytimes/{expt}/{t0}"
        # CHeck how many files are in the directory
        if os.path.exists(outputdir) and len(os.listdir(outputdir)) == 18:
            print(f"{t0} already complete.")
            continue
        print(t0)
        tmpstorage = os.getenv('PBS_JOBFS') + "/tmpstorage"
        # tmpstorage = "/scratch/nm03/ab8992/april-tmpstorage/tmpstorage/"+expt
        if os.path.exists(tmpstorage):
            shutil.rmtree(tmpstorage)
        if os.path.exists(outputdir):
            shutil.rmtree(outputdir)
        os.makedirs(outputdir)
        os.makedirs(tmpstorage)
       
        try:
            save_temporary(expt, t0.split("-")[-1], outputdir)
            save_modal(outputdir)
        except Exception as e:
            print(f"Error processing {expt} at time {t0}: {e}")
    print("Done with ",expt)
    # clear_output(wait=True)

Starting full-10
t0-25704 already complete.
t0-23544 already complete.
t0-27864 already complete.
t0-27504 already complete.
t0-28224
loading u...	done.
loading v...	done.
loading ahh...	done.
Save raw KE...
Save filtered dissipation...
t0-22200 already complete.
t0-24984 already complete.
t0-21744 already complete.
t0-28584 already complete.
t0-23184
loading u...	done.
loading v...	done.
loading ahh...	done.
Save raw KE...
Save filtered dissipation...
t0-27144 already complete.
t0-22104 already complete.
t0-24624
loading u...	done.
loading v...	done.
loading ahh...	done.
Save raw KE...
Save filtered dissipation...
t0-22464
loading u...	done.
loading v...	done.
loading ahh...	done.
Save raw KE...
Save filtered dissipation...
t0-24264 already complete.
t0-26064
loading u...	done.
loading v...	done.
loading ahh...	done.
Save raw KE...
Save filtered dissipation...
t0-25344
loading u...	done.
loading v...	done.
loading ahh...	done.
Save raw KE...
Save filtered dissipation...
t0-23904
loadi

2025-05-19 12:50:28,321 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 3.60 GiB -- Worker memory limit: 5.20 GiB
2025-05-19 12:50:35,739 - distributed.worker - ERROR - Worker stream died during communication: tcp://127.0.0.1:46373
Traceback (most recent call last):
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/tornado/iostream.py", line 861, in _read_to_buffer
    bytes_read = self.read_from_fd(buf)
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/tornado/iostream.py", line 1116, in read_from_fd
    return self.socket.recv_into(buf, len(buf))
ConnectionResetError: [Errno 104] Connection reset by peer

The above exception was the direct cause of the following ex

Error processing full-40 at time t0-3960: Attempted to run task ('sum-sum-aggregate-getitem-getitem-862b32cb709147f01cc9d606b8f97b49', 0, 0, 0, 5) on 4 different workers, but all those workers died while running it. The last worker that attempt to run the task was tcp://127.0.0.1:46323. Inspecting worker logs is often a good next step to diagnose what went wrong. For more information see https://distributed.dask.org/en/stable/killed.html.
t0-10080 already complete.
t0-9720


2025-05-19 12:51:03,814 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 3.95 GiB -- Worker memory limit: 5.20 GiB
2025-05-19 12:51:03,886 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 3.16 GiB -- Worker memory limit: 5.20 GiB
2025-05-19 12:51:03,897 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- U

loading u...	done.
loading v...	done.
loading ahh...	done.


...

Error processing smooth-40 at time t0-6840: Attempted to run task ('sum-sum-aggregate-getitem-getitem-e4f0dea7dc72d3cc34079265f0eeecdd', 0, 0, 0, 3) on 4 different workers, but all those workers died while running it. The last worker that attempt to run the task was tcp://127.0.0.1:42307. Inspecting worker logs is often a good next step to diagnose what went wrong. For more information see https://distributed.dask.org/en/stable/killed.html.
t0-4320 already complete.
t0-5400


2025-05-19 13:46:09,895 - distributed.worker.memory - WARNING - Worker is at 81% memory usage. Pausing worker.  Process memory: 4.24 GiB -- Worker memory limit: 5.20 GiB
2025-05-19 13:46:09,926 - distributed.worker.memory - WARNING - Worker is at 71% memory usage. Resuming worker. Process memory: 3.72 GiB -- Worker memory limit: 5.20 GiB


loading u...	done.
loading v...	done.
loading ahh...	done.


2025-05-19 13:47:08,000 - distributed.worker - ERROR - Worker stream died during communication: tcp://127.0.0.1:33599
Traceback (most recent call last):
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/tornado/iostream.py", line 861, in _read_to_buffer
    bytes_read = self.read_from_fd(buf)
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/tornado/iostream.py", line 1116, in read_from_fd
    return self.socket.recv_into(buf, len(buf))
ConnectionResetError: [Errno 104] Connection reset by peer

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/distributed/worker.py", line 2059, in gather_dep
    response = await get_data_from_worker(
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/distributed/worker.py", line 2

2025-05-19 13:49:33,733 - distributed.worker.state_machine - WARNING - Async instruction for <Task cancelled name="execute(('sum-sum-aggregate-getitem-getitem-bf494ad25202c156026f26d0ce635219', 0, 0, 0, 0))" coro=<Worker.execute() done, defined at /g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/distributed/worker_state_machine.py:3615>> ended with CancelledError
2025-05-19 13:49:33,974 - distributed.worker - ERROR - Failed to communicate with scheduler during heartbeat.
Traceback (most recent call last):
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/distributed/comm/tcp.py", line 225, in read
    frames_nosplit_nbytes_bin = await stream.read_bytes(fmt_size)
tornado.iostream.StreamClosedError: Stream is closed

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-package